In [1]:
import random
import numpy as np
import pandas as pd
import json
import string
import os
import csv
import random
import pickle
import geopandas as gpd

import skmob
from skmob.utils import utils, constants
from skmob.models.gravity import Gravity

In [2]:
skmob

<module 'skmob' from '/home/yuyue/miniconda3/envs/mobility/lib/python3.11/site-packages/skmob/__init__.py'>

In [3]:
db_dir = './data/houston'
tessellation = gpd.read_file(db_dir+'/tessellation.shp', dtype={"tile_ID":str})
output_areas = gpd.read_file(db_dir+'/output_areas.shp', dtype={"oa_ID":str})

In [4]:
oa_gdf = pd.read_csv(db_dir + '/processed/oa_gdf.csv.gz', dtype={'geo_code': 'str'})

flow_df = pd.read_csv(db_dir +'/processed/all/flows_oa.csv.zip', \
                      dtype={'residence': 'str', 'workplace': 'str'})

In [5]:
oa_pop = flow_df.groupby('residence').sum()
oa2pop = oa_pop.to_dict()['commuters']

/tmp/ipykernel_4191023/690305762.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  oa_pop = flow_df.groupby('residence').sum()


In [6]:
output_areas['population'] = output_areas['GEOID'].map(oa2pop)
output_areas = output_areas[output_areas['population'].notnull()]
output_areas['tot_outflow'] = output_areas['population']

In [7]:
with open(db_dir+'/processed/all/od2flow.pkl', 'rb') as f:
    od2flow = pickle.load(f)

In [8]:
from importlib.machinery import SourceFileLoader

path = './utils.py'
utils = SourceFileLoader('utils', path).load_module()
path = './data_loader.py'
dgd = SourceFileLoader('dg_data', path).load_module()

tileid2oa2features2vals, oa_gdf, flow_df, oa2pop, oa2features, od2flow, oa2centroid = utils.load_data(db_dir,
                                                                                                    'tile_ID',
                                                                                                    'geometry',
                                                                                                    'GEOID',
                                                                                                    'geometry',
                                                                                                    'geoid_o',
                                                                                                    'geoid_d',
                                                                                                    'pop_flows',
                                                                                                    'all')

./utils.py:180: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  oa_pop = flow_df.groupby('residence').sum()


In [9]:
train_data = [oa for t in pd.read_csv(db_dir + '/processed/train_tiles.csv', header=None, dtype=object)[0].values for oa
                in tileid2oa2features2vals[str(t)].keys()]
test_data = [oa for t in pd.read_csv(db_dir + '/processed/test_tiles.csv', header=None)[0].values for oa in
                tileid2oa2features2vals[str(t)].keys()]

In [10]:
np.random.seed(0)

flow_df = pd.DataFrame([{'flow': flow, 'origin': od[0], 'destination': od[1]} for od, flow in od2flow.items()])

gravity_singly = Gravity(gravity_type='singly constrained')

synth_fdf = gravity_singly.generate(output_areas,
                                   tile_id_column='GEOID',
                                   tot_outflows_column='tot_outflow',
                                   relevance_column= 'population',
                                   out_format='flows')

100%|██████████| 2139/2139 [00:03<00:00, 672.14it/s] 
/home/yuyue/miniconda3/envs/mobility/lib/python3.11/site-packages/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)


In [11]:
test_flow_df = flow_df[flow_df['origin'].isin(test_data)]

merge_flow_df = pd.merge(
    left=test_flow_df, 
    right=synth_fdf,
    how='left',
    left_on=['origin', 'destination'],
    right_on=['origin', 'destination'],
)

merge_flow_df = merge_flow_df[(merge_flow_df['flow_x'].notnull()) & (merge_flow_df['flow_y'].notnull())]

In [12]:
from scipy import stats
from scipy.spatial import distance
from skmob.measures.evaluation import nrmse, pearson_correlation, kullback_leibler_divergence, r_squared, rmse

def common_part_of_commuters(values1, values2, numerator_only=False):
    if numerator_only:
        tot = 1.0
    else:
        tot = (np.sum(values1) + np.sum(values2))
    if tot > 0:
        return 2.0 * np.sum(np.minimum(values1, values2)) / tot
    else:
        return 0.0

target_flow_list = merge_flow_df['flow_x'].values
predicted_flow_list = merge_flow_df['flow_y'].values

print("CPC", common_part_of_commuters(target_flow_list, predicted_flow_list))

print("pearson", stats.pearsonr(target_flow_list, predicted_flow_list))

print("JSD", distance.jensenshannon(target_flow_list, predicted_flow_list))

print("RMSE", rmse(target_flow_list, predicted_flow_list))

print("NRMSE", rmse(target_flow_list, predicted_flow_list) / (max(target_flow_list) - min(target_flow_list)))

CPC 0.46045260017622425
pearson PearsonRResult(statistic=0.38622716440431476, pvalue=0.0)
JSD 0.45169923590477074
RMSE 245.48516466171512
NRMSE 0.008340190414544918
